In [ ]:
# Import statements (standard)
import math
import time
import numpy as np
import pandas as pd
import datetime as dt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# Import statements (custom)
import helper_functions as hf

# Import statements (RF)
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
bitcoin_market_info = pd.read_csv('/Users/zhaocai/Downloads/courses/MSBD5013_Statistical_Prediction/data/BTC-USD_T.csv',index_col='time')

In [2]:
bitcoin_market_info


time    close     high      low     open    volume
0        2014-12-01 05:33:00   300.00   300.00   300.00   300.00  0.010000
1        2014-12-01 05:34:00   300.00   300.00   300.00   300.00  0.000000
2        2014-12-01 05:35:00   300.00   300.00   300.00   300.00  0.000000
3        2014-12-01 05:36:00   300.00   300.00   300.00   300.00  0.000000
4        2014-12-01 05:37:00   300.00   300.00   300.00   300.00  0.000000
5        2014-12-01 05:38:00   300.00   300.00   300.00   300.00  0.000000
6        2014-12-01 05:39:00   300.00   300.00   300.00   300.00  0.000000
7        2014-12-01 05:40:00   300.00   300.00   300.00   300.00  0.010000
8        2014-12-01 05:41:00   300.00   300.00   300.00   300.00  0.000000
9        2014-12-01 05:42:00   300.00   300.00   300.00   300.00  0.000000
10       2014-12-01 05:43:00   300.00   300.00   300.00   300.00  0.000000
11       2014-12-01 05:44:00   300.00   300.00   300.00   300.00  0.000000
12       2014-12-01 05:45:00   300.00   300.00   300.00   300.00  0.000000
13       2014-12-01 05:46:00   300.00   300.00   300.00   300.00  0.000000
14       2014-12-01 05:47:00   300.00   300.00   300.00   300.00  0.000000
15       2014-12-01 05:48:00   300.00   300.00   300.00   300.00  0.000000
16       2014-12-01 05:49:00   300.00   300.00   300.00   300.00  0.000000
17       2014-12-01 05:50:00   300.00   300.00   300.00   300.00  0.000000
18       2014-12-01 05:51:00   300.00   300.00   300.00   300.00  0.000000
19       2014-12-01 05:52:00   300.00   300.00   300.00   300.00  0.000000
20       2014-12-01 05:53:00   300.00   300.00   300.00   300.00  0.000000
21       2014-12-01 05:54:00   300.00   300.00   300.00   300.00  0.000000
22       2014-12-01 05:55:00   300.00   300.00   300.00   300.00  0.000000
23       2014-12-01 05:56:00   300.00   300.00   300.00   300.00  0.000000
24       2014-12-01 05:57:00   300.00   300.00   300.00   300.00  0.000000
25       2014-12-01 05:58:00   300.00   300.00   300.00   300.00  0.000000
26       2014-12-01 05:59:00   300.00   300.00   300.00   300.00  0.000000
27       2014-12-01 06:00:00   300.00   300.00   300.00   300.00  0.000000
28       2014-12-01 06:01:00   300.00   300.00   300.00   300.00  0.000000
29       2014-12-01 06:02:00   300.00   300.00   300.00   300.00  0.000000
...                      ...      ...      ...      ...      ...       ...
1984500  2018-09-09 08:33:00  6191.04  6191.04  6191.04  6191.04  0.449283
1984501  2018-09-09 08:34:00  6191.04  6191.04  6191.03  6191.03  0.437767
1984502  2018-09-09 08:35:00  6191.04  6191.04  6191.03  6191.04  0.126160
1984503  2018-09-09 08:36:00  6191.04  6191.04  6191.03  6191.04  0.312845
1984504  2018-09-09 08:37:00  6192.70  6192.71  6191.03  6191.04  8.405736
1984505  2018-09-09 08:38:00  6191.04  6191.05  6191.04  6191.05  0.660315
1984506  2018-09-09 08:39:00  6191.05  6191.05  6191.04  6191.04  1.106928
1984507  2018-09-09 08:40:00  6191.04  6191.05  6191.04  6191.05  0.528902
1984508  2018-09-09 08:41:00  6191.04  6191.05  6191.04  6191.05  0.340553
1984509  2018-09-09 08:42:00  6191.05  6191.05  6191.05  6191.05  0.093486
1984510  2018-09-09 08:43:00  6191.04  6191.05  6191.04  6191.05  1.411334
1984511  2018-09-09 08:44:00  6191.05  6191.05  6191.04  6191.04  0.344474
1984512  2018-09-09 08:45:00  6191.05  6191.05  6191.04  6191.04  2.862518
1984513  2018-09-09 08:46:00  6191.06  6191.06  6191.04  6191.04  1.353602
1984514  2018-09-09 08:47:00  6191.05  6191.06  6191.05  6191.06  0.245435
1984515  2018-09-09 08:48:00  6191.06  6191.06  6191.05  6191.05  0.035564
1984516  2018-09-09 08:49:00  6191.05  6191.06  6191.05  6191.06  0.984535
1984517  2018-09-09 08:50:00  6191.06  6191.06  6191.05  6191.06  6.860632
1984518  2018-09-09 08:51:00  6191.05  6191.06  6191.05  6191.06  1.162322
1984519  2018-09-09 08:52:00  6191.06  6191.06  6191.05  6191.06  0.561030
1984520  2018-09-09 08:53:00  6188.44  6191.06  6186.96  6191.06  0.495437
1984521  2018-09-09 08:54:00  6188.44  6188.44  6

In [3]:
bitcoin_market_info.head()


time  close   high    low   open  volume
0  2014-12-01 05:33:00  300.0  300.0  300.0  300.0    0.01
1  2014-12-01 05:34:00  300.0  300.0  300.0  300.0    0.00
2  2014-12-01 05:35:00  300.0  300.0  300.0  300.0    0.00
3  2014-12-01 05:36:00  300.0  300.0  300.0  300.0    0.00
4  2014-12-01 05:37:00  300.0  300.0  300.0  300.0    0.00

In [4]:
bitcoin_market_info.tail()

time    close     high      low     open    volume
1984525  2018-09-09 08:58:00  6192.78  6192.78  6192.77  6192.78  0.735574
1984526  2018-09-09 08:59:00  6192.78  6192.78  6192.77  6192.77  0.221675
1984527  2018-09-09 09:00:00  6192.78  6192.78  6192.77  6192.78  0.852274
1984528  2018-09-09 09:01:00  6192.77  6192.78  6192.77  6192.77  1.563542
1984529  2018-09-09 09:02:00  6192.77  6192.78  6192.77  6192.78  0.589552

In [ ]:
bitcoin_market_info.columns